In [ ]:
import torch
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

DATA_DIR = "/home/borth/2d-gaussian-splatting/data/DTU"
SCAN_ID = "scan37"


# Camera Data

In [ ]:
cameras = np.load(Path(DATA_DIR, SCAN_ID, "cameras.npz"))
print(list(cameras.keys()))

In [ ]:
ply_path = Path(DATA_DIR, SCAN_ID, "points.ply")
visualize_scene(ply_path)

In [ ]:
path = "/home/borth/2d-gaussian-splatting/output/input.ply"
visualize_scene(ply_path)

In [ ]:
import torch

idx = 0
depth_map = torch.load(Path(DATA_DIR, SCAN_ID, f"depths/{idx:04}.pt"), weights_only=False)
print(depth_map.keys())
print(depth_map["depth"], depth_map["depth"].shape)
print(depth_map["coord"], depth_map["coord"].shape)
print(depth_map["error"], depth_map["error"].shape)
print(depth_map["weight"], depth_map["weight"].shape)

In [ ]:
from PIL import Image
img = Image.open("/home/borth/2d-gaussian-splatting/data/DTU/scan24/mask/000.png")
img

In [ ]:
from PIL import Image
img = Image.open("/home/borth/2d-gaussian-splatting/data/DTU/scan24/images/0000.png")
np.asarray(img).shape

In [ ]:
import trimesh
import open3d as o3d

# Load the mesh with trimesh
mesh = trimesh.load_mesh("/home/borth/2d-gaussian-splatting/logs/2025-02-13/15-00-53/train/ours_100/fuse_cull.ply")


In [ ]:
import open3d as o3d

# Load the mesh
path = "/home/borth/2d-gaussian-splatting/logs/2025-02-13/15-00-53/train/ours_100/fuse_cull.ply"
mesh = o3d.io.read_triangle_mesh(path)

# Compute normals for better shading
# mesh.compute_vertex_normals()

# Visualize the mesh
o3d.visualization.draw_plotly([mesh], window_name="Mesh Viewer")


In [ ]:
from hydra import compose, initialize
from lib.scene import Scene, GaussianModel
from lib.utils.mesh_utils import to_cam_open3d
import open3d as o3d

with initialize(config_path="./conf", version_base=None):
    cfg = compose(config_name="train", overrides=[])

cfg.dataset.model_path = ""
gaussians = GaussianModel(cfg.dataset.sh_degree)
scene = Scene(cfg.dataset, gaussians)
cameras = to_cam_open3d(scene.getTrainCameras())


In [ ]:
import open3d as o3d

# load the mesh
path = "/home/borth/2d-gaussian-splatting/logs/2025-02-13/15-00-53/train/ours_100/fuse_cull.ply"
mesh = o3d.io.read_triangle_mesh(path)
mesh.compute_vertex_normals()  # Optionally compute normals for shading

# Create an offscreen renderer
renderer = o3d.visualization.rendering.OffscreenRenderer(800, 600)


# Create material for the mesh
material = o3d.visualization.rendering.MaterialRecord()
material.base_color = (0.8, 0.8, 0.8, 1.0)  # Example: Light gray color
material.shader = "defaultLit"

# Add the mesh to the scene
renderer.scene.add_geometry("mesh", mesh, material)

# Set up the camera
# scene.camera.intrinsic = cameras[0].intrinsic
# scene.camera.extrinsic = cameras[0].extrinsic

# Add custom lighting
light = o3d.visualization.rendering.DirectionalLight(color=(1.0, 1.0, 1.0), intensity=1.0)
renderer.scene.add_light(light, pose=[[1, 0, 0, 2], [0, 1, 0, 2], [0, 0, 1, 2], [0, 0, 0, 1]])


# Set camera pose (if you have the camera pose)
eye = [0, 0, 2]  # Camera position
center = mesh.get_center()  # Look at the mesh's center
up = [0, 1, 0]  # 'Up' direction

# You can directly modify the camera's pose
renderer.scene.camera.set_camera_pose(center, eye, up)

# Render the scene to an image
img = renderer.render_to_image()

# Save the image
o3d.io.write_image("rendered_mesh_with_lighting.png", img)

# Free resources
renderer = None
from PIL import Image
import numpy as np
Image.fromarray(np.asarray(img))


In [2]:
import open3d as o3d

# Load the mesh
path = "/home/borth/2d-gaussian-splatting/logs/2025-02-13/15-00-53/train/ours_100/fuse_cull.ply"
mesh = o3d.io.read_triangle_mesh(path)
mesh.compute_vertex_normals()  # Optionally compute normals for shading

# Create an offscreen renderer
renderer = o3d.visualization.rendering.OffscreenRenderer(800, 600)

# Create material for the mesh
material = o3d.visualization.rendering.MaterialRecord()
material.base_color = (0.8, 0.8, 0.8, 1.0)  # Example: Light gray color
material.shader = "defaultLit"

# Add the mesh to the scene
renderer.scene.add_geometry("mesh", mesh, material)

# Set up the camera
bounds = mesh.get_axis_aligned_bounding_box()
center = bounds.get_center()
eye = center + [0, 0, 2]  # Move the camera back
renderer.scene.camera.look_at(center, eye, [0, 1, 0])

# Add custom lighting
# light = o3d.visualization.rendering.DirectionalLight(color=(1.0, 1.0, 1.0), intensity=1.0)
# renderer.scene.add_light(light, pose=[[1, 0, 0, 2], [0, 1, 0, 2], [0, 0, 1, 2], [0, 0, 0, 1]])

# Render the scene to an image
img = renderer.render_to_image()

# Save the image
o3d.io.write_image("rendered_mesh_with_lighting.png", img)

# Free resources
renderer = None

: 

In [ ]:
np.asarray(mesh.vertices)

In [ ]:
center

In [ ]:
import trimesh
import pyrender
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"  # or "egl" if using a GPU


# Create a simple sphere mesh
mesh = trimesh.load_mesh("/home/borth/2d-gaussian-splatting/logs/2025-02-13/10-15-18/train/ours_10/fuse_cull.ply") 

# Convert to pyrender format
mesh_pyrender = pyrender.Mesh.from_trimesh(mesh)

# Create a scene
scene = pyrender.Scene()
scene.add(mesh_pyrender)

# Add a camera
camera = pyrender.PerspectiveCamera(yfov=np.pi / 2.0)
scene.add(camera, pose=np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 5],
    [0, 0, 0, 1]
]))

# Add a light source
# light = pyrender.DirectionalLight(color=np.ones(3), intensity=2.0)
# scene.add(light, pose=np.array([
#     [1, 0, 0, 1],
#     [0, 1, 0, 1],
#     [0, 0, 1, 1],
#     [0, 0, 0, 1]
# ]))

# Create a renderer
r = pyrender.OffscreenRenderer(viewport_width=800, viewport_height=600)

# Render the scene
color, depth = r.render(scene)
r.delete()


# Display the rendered image
plt.imshow(color)
plt.axis('off')
plt.show()


In [ ]:
import trimesh

# Load the mesh from a file
mesh = trimesh.load_mesh("/home/borth/2d-gaussian-splatting/logs/2025-02-13/10-15-18/train/ours_10/fuse_cull.ply")

import pyrender
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"  # or "egl" if using a GPU
# Convert to a Pyrender-compatible format
mesh_pyrender = pyrender.Mesh.from_trimesh(mesh)

# Create a scene
scene = pyrender.Scene()
scene.add(mesh_pyrender)

# Add better lighting (Directional Light)
light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=3.0)
scene.add(light)

# Add a camera for a better view
camera = pyrender.PerspectiveCamera(yfov=0.7)
scene.add(camera, pose=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 2],  # Move the camera back to see the model
                        [0, 0, 0, 1]])

# Add a camera for better viewing
camera = pyrender.PerspectiveCamera(yfov=0.7)
scene.add(camera, pose=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 2],  # Move camera back to view model
                        [0, 0, 0, 1]])

# Render interactively
pyrender.Viewer(scene, use_raymond_lighting=True)  # Opens an interactive window